In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
%matplotlib inline
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os
import folium

## Read  CHILE GEM SARA Model (csv format)

In [3]:
path = '/Users/pittore/Documents/PROJECTS/BMBF/RIESGOS/data/GEM_sara_chile/chl-l3-exposure'
infile = 'chl-l3-exposure_total.csv'
sara_data = pd.read_csv(os.path.join(path,infile))

#clean 'name' and use it as ID
sara_data['name'] = sara_data.Region.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
#save all names in an array
sara_names = sara_data['name'].unique()

#group data based on 'name'
datagrp = sara_data.groupby('name')

## Read Chile admin_3 areas from GADM

In [4]:
chile_map = '/Users/pittore/Documents/PROJECTS/BMBF/RIESGOS/data/GADM_chile/gadm36_CHL_3.json'
chile_map_gpd = gpd.read_file(chile_map)
chile_map_gpd['name'] = chile_map_gpd.NAME_3.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
len(chile_map_gpd)

302

## Associate exposure model for each admin area and change geometry type to MultiPolygon

In [5]:
from shapely.geometry import MultiPolygon

outmodel = chile_map_gpd[['name','GID_3','geometry']]
outmodel['expo'] = ""
#outmodel.head()
outmodel = outmodel.set_index('name')

for gcname,gc in outmodel.iterrows():
    if (gcname in sara_names):
            testgrp = datagrp.get_group(gcname)
            #add initial damage state - by default Null Damage (D0)
            testgrp['Damage'] = 'D0'
            outmodel.at[gcname,'expo'] = testgrp.to_json()
    #convert polygons into multipolygons to avoid errors in file output
    if (gc.geometry.geom_type == 'Polygon'):
        outmodel.at[gcname,'geometry'] = MultiPolygon([gc.geometry])

outmodel.columns = ['gid', 'geometry', 'expo']
outmodel = outmodel.reset_index()
outmodel.head()

,name,gid,geometry,expo
0,Aisen,CHL.1.1.1_1,(POLYGON ((-73.49375152587885 -44.965415954589...,"{""id"":{""5709"":""AREA # 11201"",""5710"":""AREA # 11..."
1,Cisnes,CHL.1.1.2_1,(POLYGON ((-73.31124877929682 -45.111248016357...,"{""id"":{""5729"":""AREA # 11202"",""5730"":""AREA # 11..."
2,Guaitecas,CHL.1.1.3_1,(POLYGON ((-73.49153137207031 -43.711528778076...,"{""id"":{""5747"":""AREA # 11203"",""5748"":""AREA # 11..."
3,Cochrane,CHL.1.2.1_1,(POLYGON ((-72.42082214355469 -47.743259429931...,"{""id"":{""5757"":""AREA # 11301"",""5758"":""AREA # 11..."
4,O'Higgins,CHL.1.2.2_1,(POLYGON ((-73.17006683349609 -49.224048614501...,"{""id"":{""5775"":""AREA # 11302"",""5776"":""AREA # 11..."


### Drop geocells with empty exposure 

In [ ]:
blacklist  = outmodel[outmodel.expo ==''].index
outmodel1 = outmodel.drop(blacklist)

## Save to GEOPACKAGE 

In [25]:
#ioschema = gpd.io.file.infer_schema(outmodel)
#ioschema['geometry']='MultiPolygon'
#ioschema
outmodel1.to_file("output.gpkg", driver="GPKG")#,schema = ioschema)

## Display the boundary of the model

In [ ]:
map_osa = folium.Map(location=[-32,-71],tiles='Stamen Terrain',zoom_start=10)
map_osa.choropleth(geo_data=outmodel1,fill_opacity=0.3)
#map_osa.choropleth(geo_data=chile_map,fill_opacity=0.3)
#folium.Marker([-32, -71], popup='test').add_to(map_osa)
map_osa

In [ ]:
#map_osa = folium.Map(location=[df.iloc[event_index].latitude,df.iloc[event_index].longitude])
#map_osm.create_map(path='osm.html')
#map_osa.save('testfolium.html')
#map_osa